## Install & Import necessary libreries

In [ ]:
# install 
!pip install pandas
!pip install matplotlib

In [ ]:
# import 
import pandas as pd
import matplotlib.pyplot as plt
import os

### Task 1: merging the 12 months of sales data into a singel file

In [ ]:
files_names = os.listdir('./sales_data')
data_frames = []

for file_name in files_names:
  df = pd.read_csv(f'./sales_data/{file_name}')
  data_frames.append(df)

df = pd.concat(data_frames)
df.head()

## Task 2: clean up the data

In [ ]:
# display NaN rows
nan_df = df[df.isna().any(axis=1)]
nan_df.head()

In [ ]:
# delete rows whos contain NaN value
df = df.dropna(how='all')

In [ ]:
# display rows whos contain or value
or_rows = df[df['Order Date'].str[0:2] == 'Or']
or_rows.head()

In [ ]:
# delete Or rows
df = df[df['Order Date'].str[0:2]  != 'Or']

## Task 3: add the month column

In [ ]:
df['Month'] = df['Order Date'].str[0:2]
df['Month'] = df['Month'].astype('int32')
df.head()

## Qst 1: what was the best sales for month ? and how much earned that month ?

## Task 1: add sells column

In [ ]:
df['sales'] = df['Price Each'].astype('float32') * df['Quantity Ordered'].astype('int32')
df.head()

## Task 2: display the bar plot

In [ ]:
sales_by_month = df.groupby('Month').sum()
months = range(1,13)
plt.bar(months,sales_by_month['sales'])
plt.xticks(months)
plt.xlabel('Months number')
plt.ylabel('Sales in USD $')
plt.show()

In [ ]:
sales_by_month


now we can see that the best sales for month is the month number 12 which is december and he earned 4613443.50

## Qst 2: what city had the highest number of sales

### Task 1: add city column

In [ ]:
df['City'] = df['Purchase Address'].apply(lambda x: x.split(',')[1])
df.head()

In [ ]:
sales_by_city = df.groupby('City').sum()
cities = df['City'].unique() 
plt.bar(cities,sales_by_city['sales'])
plt.xticks(cities,rotation='vertical',size=8)
plt.xlabel('Cities')
plt.ylabel('Sales in USD $')
plt.show()

In [ ]:
sales_by_city

now we can see that the city whos had the highest number of sales is Atlanta

Qst 3: What time should we display advertisements to maximize likelejood of custumer's buying product

In [ ]:
# convert Order Date values to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Hour'] = df['Order Date'].dt.hour
df.head()

In [ ]:
hours = df['Hour'].unique()
sales_by_hour = df.groupby(['Hour']).count()
plt.plot(sales_by_hour['sales'])
plt.xticks(hours,rotation='vertical',size=8)
plt.grid()
plt.xlabel('hours')
plt.ylabel('Sales in USD $')
plt.show()
sales_by_hour.head()

Answer : 20h and 12h

## Qst 4: what products are most often sold together ?

### Task 1: get duplicated order id & group the products

In [ ]:
duplicated_id = df[df['Order ID'].duplicated(keep=False)]
duplicated_id['Grouped'] = duplicated_id.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))
grouped_products = duplicated_id[['Order ID','Grouped']].drop_duplicates()
grouped_products.head()

### Task 2: get most product sold together

In [ ]:
from itertools import combinations
from collections import Counter

count = Counter()

for row in grouped_products['Grouped']:
  row_list = row.split(',')
  count.update(Counter(combinations(row_list,2)))

count.most_common(10)